In [ ]:
# =========================
# Cell 1: 4×4 projector onto the low-energy doublet + τ-operators (all as 4×4)
# =========================
import sympy as sp
I = sp.I
kron = sp.kronecker_product

# --- Parameters in H0 ---
Cx, J = sp.symbols("C_x J", real=True)

# --- Parameters in V (kept for Cell 2) ---
m, Kx = sp.symbols("m K_x", real=True)

# --- Long-wavelength placeholders (commutative): treat ∂ and A as symbols ---
dx, dy = sp.symbols("d_x d_y", commutative=True)        # stand-ins for ∂x, ∂y
Aparx, Apary = sp.symbols("Aparx Apary", commutative=True)  # A^||_x, A^||_y

# transverse SU(2) connection components: A^⊥_j = (A^⊥_{j,1}, A^⊥_{j,2})
Apx1, Apx2, Apy1, Apy2 = sp.symbols(
    "Aperp_x1 Aperp_x2 Aperp_y1 Aperp_y2", commutative=True
)

# --- Pauli matrices (spin σ and sublattice η) ---
sx = sp.Matrix([[0, 1],[1, 0]])
sy = sp.Matrix([[0,-I],[I, 0]])
sz = sp.Matrix([[1, 0],[0,-1]])
s0 = sp.eye(2)

etax, etay, etaz, eta0 = sx, sy, sz, s0

Id4 = sp.eye(4)

# --- Unperturbed 4×4 Hamiltonian defining the doublet ---
# H0 = Cx ηx - J ηz σz
H0 = Cx*kron(etax, s0) - J*kron(etaz, sz)
E0 = sp.sqrt(Cx**2 + J**2)

# Projector onto the negative-energy (-E0) subspace (rank-2 projector)
P = (Id4 - H0/E0)/2

# Mixing-angle shorthands used in the appendix
s_th = sp.simplify(Cx/E0)   # sin(vartheta)
c_th = sp.simplify(J/E0)    # cos(vartheta)

# --- τ operators as 4×4 matrices supported only on the P-subspace ---
# (This is exactly the appendix definition: τ = P ( ... ) P)
tau0 = sp.simplify(P*kron(eta0, s0)*P)   # = P
taux = sp.simplify(P*kron(etax, sx)*P)
tauy = sp.simplify(P*kron(etax, sy)*P)
tauz = sp.simplify(P*kron(eta0, sz)*P)

# --- Helpers ---
def msimplify(M):
    return M.applyfunc(lambda e: sp.simplify(sp.together(e)))

def assert_mat_equal(A, B, label=""):
    D = msimplify(A - B)
    if any(el != 0 for el in D):
        raise AssertionError(f"{label} FAILED.\nDifference =\n{D}")

def assert_zero(A, label=""):
    assert_mat_equal(A, sp.zeros(*A.shape), label=label)

# --- Sanity checks: projector properties and τ algebra inside the P-subspace ---
assert_zero(msimplify(P*P - P), "Projector: P^2 != P")
assert_zero(msimplify(tau0 - P), "tau0 != P")

# Optional: check the key projection identities used later (Appendix B20-B22 style)
assert_zero(P*kron(eta0, sx)*P + s_th*taux, "P σx P != -sinθ τx")
assert_zero(P*kron(eta0, sy)*P + s_th*tauy, "P σy P != -sinθ τy")
assert_zero(P*kron(etax, s0)*P + s_th*tau0, "P ηx P != -sinθ τ0")
assert_zero(P*kron(eta0, sz)*P - tauz,      "P σz P != τz")  

print("Cell 1 OK: built 4×4 projector P and 4×4 τ-operators.")

Cell 1 OK: built 4×4 projector P and 4×4 τ-operators (plus basic identities).


In [2]:
# =========================
# Cell 2: First two "cells" from the 4×4 model:
#         compute P V P directly and prove it equals the expected projected form
#         (ONLY the D^2 terms: -η0 D^2/(2m) - ηx D^2*(Kx/2); no Kz terms here)
# =========================

# --- Spin-space covariant derivatives ---
# D||_j = ∂j + i A^||_j σz/2
Dparx = dx*s0 + I*Aparx*sz/2
Dpary = dy*s0 + I*Apary*sz/2

# T_j  = i (A^⊥_j · σ⊥)/2  with σ⊥ = (σx, σy)
Tx = I*(Apx1*sx + Apx2*sy)/2
Ty = I*(Apy1*sx + Apy2*sy)/2

# Full SU(2) covariant derivatives
Dx = Dparx + Tx
Dy = Dpary + Ty

# D^2 = Dx^2 + Dy^2 in spin space (2×2)
D2 = sp.simplify(Dx*Dx + Dy*Dy)

# --- The 4×4 perturbation containing the first two "cells" ---
# V = -(1/2m) η0 D^2  - (Kx/2) ηx D^2
V = -(1/(2*m))*kron(eta0, D2) - (Kx/2)*kron(etax, D2)

# --- Project to the low-energy doublet (still as a 4×4 operator supported on P) ---
PVP = msimplify(P*V*P)

# -------------------------------------------------------------------------
# Build the EXPECTED projected operator (first two cells) as a 4×4 object.
#
# Key trick: build the *projected* longitudinal covariant derivative directly
# using τz (4×4), i.e. inside the doublet σz → τz and identity → τ0 (=P).
#
# D||_x,eff = ∂x τ0 + i A^||_x τz/2   (as a 4×4 operator in the P-subspace)
# -------------------------------------------------------------------------
Dpar_eff_x = dx*tau0 + I*Aparx*tauz/2
Dpar_eff_y = dy*tau0 + I*Apary*tauz/2

# Metric from transverse gauge field: g_jk = A^⊥_j · A^⊥_k
gxx = Apx1**2 + Apx2**2
gyy = Apy1**2 + Apy2**2

# The linear transverse piece uses B = A^⊥_x ∂x + A^⊥_y ∂y, dotted into τ⊥.
# Components along the internal "⊥" directions:
B1 = Apx1*dx + Apy1*dy
B2 = Apx2*dx + Apy2*dy
B_dot_tau = B1*taux + B2*tauy

# Coefficients that the appendix derives:
# kinetic/metric coefficient:  (1/(2m) - Kx sinθ / 2)
mu = sp.simplify(sp.Rational(1,2)/m - sp.Rational(1,2)*Kx*s_th)

# SOC-like coefficient:  - i (Kx/2 - sinθ/(2m))
lam = sp.simplify(-I*(sp.Rational(1,2)*Kx - sp.Rational(1,2)*s_th/m))

# Expected projected operator (first two cells only):
#   -mu * [ (D||_x)^2 + (D||_y)^2 ]  +  mu*(gxx+gyy) τ0/4   +  lam * (B·τ⊥)
PVP_expected = msimplify(
    -mu*(Dpar_eff_x*Dpar_eff_x + Dpar_eff_y*Dpar_eff_y)
    + mu*(gxx + gyy)*tau0/4
    + lam*B_dot_tau
)

# Force-support on the subspace (harmless, but makes intent explicit)
PVP_expected = msimplify(tau0*PVP_expected*tau0)

# --- PROOF: direct 4×4 projection equals the expected "first two cells" form ---
assert_mat_equal(PVP, PVP_expected, label="First two cells: 4×4 projection check")

print("Cell 2 OK: P V P from the 4×4 model matches the expected first-two-cells projected form.")

Cell 2 OK: P V P from the 4×4 model matches the expected first-two-cells projected form.


In [4]:
# =========================
# Cell 2b: g-wave rank-4 term (B26) -> 4×4 projection P V_g P -> (B27)
# =========================
import sympy as sp
I = sp.I

# --- Require Cell 1 objects (projector + tau operators + Pauli) ---
needed = ["P","tau0","tauz","s_th","c_th","kron","sx","sy","sz","s0","etaz"]
missing = [x for x in needed if x not in globals()]
if missing:
    raise RuntimeError("Missing symbols from Cell 1: " + ", ".join(missing))

# --- New coupling for g-wave operator ---
Kzg = sp.symbols("Kz_g", commutative=True)

# --- Long-wavelength placeholders (commutative) ---
dx, dy = sp.symbols("d_x d_y", commutative=True)                 # stand-ins for ∂x, ∂y
Aparx, Apary = sp.symbols("Aparx Apary", commutative=True)       # A^||_x, A^||_y
Apx1, Apx2, Apy1, Apy2 = sp.symbols(
    "Aperp_x1 Aperp_x2 Aperp_y1 Aperp_y2", commutative=True
)

# --- Spin-space longitudinal covariant derivatives D^||_j (2×2) ---
Dparx = dx*s0 + I*Aparx*sz/2
Dpary = dy*s0 + I*Apary*sz/2
Dpar  = {'x': Dparx, 'y': Dpary}

# --- Spin-space transverse insertions A^⊥_j · σ_⊥ (2×2) ---
Aperp_dot_sigma = {
    'x': Apx1*sx + Apx2*sy,
    'y': Apy1*sx + Apy2*sy,
}

# --- Texture metric g_jk = A^⊥_j · A^⊥_k (scalars) ---
gxx = Apx1**2 + Apx2**2
gyy = Apy1**2 + Apy2**2
gxy = Apx1*Apy1 + Apx2*Apy2
g = {('x','x'): gxx, ('y','y'): gyy, ('x','y'): gxy, ('y','x'): gxy}

def msimplify(M):
    return M.applyfunc(lambda e: sp.simplify(sp.together(e)))

def assert_mat_equal(A, B, label=""):
    D = msimplify(A - B)
    if any(el != 0 for el in D):
        raise AssertionError(f"{label} FAILED.\nDifference =\n{D}")

# -----------------------------------------------------------------------------
# Weyl-symmetrized quartic product W[DjDkDlDm] truncated to O((A⊥)^2)
# using Eq. (B9). This is the *only* approximation in this cell.
# -----------------------------------------------------------------------------
def W4_B9(seq):
    """
    seq: list like ['x','y','y','y'] meaning W[ D_x D_y D_y D_y ].
    Returns a 2×2 spin-space matrix implementing Eq. (B9) to O((A⊥)^2).
    """
    # (B9) leading longitudinal piece
    out = sp.eye(2)
    for a in seq:
        out = out * Dpar[a]

    # (B9) linear term:  (i/8) Σ_r [ (A⊥_{a_r}·σ⊥) Π_{s≠r} D||_{a_s} + Π_{s≠r} D||_{a_s} (A⊥_{a_r}·σ⊥) ]
    lin = sp.zeros(2)
    for r, ar in enumerate(seq):
        prod_excl = sp.eye(2)
        for s, a in enumerate(seq):
            if s != r:
                prod_excl = prod_excl * Dpar[a]
        Aop = Aperp_dot_sigma[ar]
        lin += Aop*prod_excl + prod_excl*Aop
    out += I*lin/8

    # (B9) metric/scalar term:  -(1/16) Σ_{r<s} g_{a_r a_s} σ0 Π_{t≠r,s} D||_{a_t}
    met = sp.zeros(2)
    for r in range(4):
        for s in range(r+1, 4):
            prod_excl = sp.eye(2)
            for t, a in enumerate(seq):
                if t not in (r, s):
                    prod_excl = prod_excl * Dpar[a]
            met += g[(seq[r], seq[s])] * prod_excl
    out += -(sp.Rational(1,16)) * met * s0

    return msimplify(out)

# -----------------------------------------------------------------------------
# Build the g-wave operator inside the 4×4 Hamiltonian:
#   V_g = + (Kzg/4!) ηz W[ Dx Dy (Dy^2 - Dx^2) ]
# with the Weyl truncation implemented via:
#   W[DxDy(Dy^2 - Dx^2)] = W[xyyy] - W[xyxx].
# -----------------------------------------------------------------------------
W_xyyy = W4_B9(['x','y','y','y'])
W_xyxx = W4_B9(['x','y','x','x'])
W_g    = msimplify(W_xyyy - W_xyxx)

Vg_4x4 = (Kzg/sp.factorial(4)) * kron(etaz, W_g)

# Project: P V_g P (still 4×4, supported on the P-subspace)
PVP_g = msimplify(P * Vg_4x4 * P)

# -----------------------------------------------------------------------------
# Expected result (B27): P ηz W[...] P = cosθ τz { ... }
# Therefore P Vg P = (Kzg/4!) cosθ τz { ... }.
# The bracket uses the projected longitudinal D|| operators:
#   D||_x,eff = ∂x τ0 + i A||_x τz/2, etc.
# -----------------------------------------------------------------------------
Dpar_eff_x = dx*tau0 + I*Aparx*tauz/2
Dpar_eff_y = dy*tau0 + I*Apary*tauz/2

bracket = (
    Dpar_eff_x*Dpar_eff_y*(Dpar_eff_y*Dpar_eff_y - Dpar_eff_x*Dpar_eff_x)
    - sp.Rational(3,16) * (
        gxy*(Dpar_eff_y*Dpar_eff_y - Dpar_eff_x*Dpar_eff_x)
        + (gyy - gxx)*Dpar_eff_x*Dpar_eff_y
    )
)

PVP_g_expected = msimplify((Kzg/sp.factorial(4)) * c_th * tauz * bracket)

# # Confine explicitly to P-subspace (optional but clarifies intent)
# PVP_g_expected = msimplify(tau0 * PVP_g_expected * tau0)

# --- PROOF ---
assert_mat_equal(PVP_g, PVP_g_expected, label="g-wave 4×4 projection (B26)->(B27)")

print("OK: g-wave term P V_g P matches Eq. (B27) (and hence the g-wave line in Eq. (B28)) to O((A⊥)^2).")

OK: g-wave term P V_g P matches Eq. (B27) (and hence the g-wave line in Eq. (B28)) to O((A⊥)^2).
